Importar librerias

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

definir la URL y realizar conexion esperando respuesta con estatus 200

In [2]:
url='https://www.eltiempo.com/'
tiempo=requests.get(url)
tiempo.status_code

200

Crear sopa de letras con la url

In [3]:
tiempo_soup=BeautifulSoup(tiempo.text,'lxml')
type(tiempo_soup)

bs4.BeautifulSoup

Creando lista con secciones

In [4]:
secciones=tiempo_soup.find('ul',attrs={'class':'default-menu'}).find_all('li')

# Usando list comprehension
#lista_notas = [url + article.a.get('href') for article in articles.find_all('div', attrs={'class': 'article-item__content'}) if article.a]

#Lista Secciones
nombre_secciones=[nseccion.a.get_text('href') for nseccion in secciones]
link_secciones=[url + lseccion.a.get('href') for lseccion in secciones]

In [5]:
nombre_secciones

['Opinión',
 'Colombia',
 'Bogotá',
 'Internacional',
 'Política',
 'Justicia',
 'Economía',
 'Deportes',
 'Cultura',
 'Tecnología',
 'Vida',
 'Unidad Investigativa',
 'Salud']

In [6]:
link_secciones

['https://www.eltiempo.com//opinion',
 'https://www.eltiempo.com//colombia',
 'https://www.eltiempo.com//bogota',
 'https://www.eltiempo.com//mundo',
 'https://www.eltiempo.com//politica',
 'https://www.eltiempo.com//justicia',
 'https://www.eltiempo.com//economia',
 'https://www.eltiempo.com//deportes',
 'https://www.eltiempo.com//cultura',
 'https://www.eltiempo.com//tecnosfera',
 'https://www.eltiempo.com//vida',
 'https://www.eltiempo.com//unidad-investigativa',
 'https://www.eltiempo.com//salud']

In [7]:
secciones_data = {'nombre_seccion': nombre_secciones, 'Enlace': link_secciones}
secciones_df = pd.DataFrame(secciones_data)
#secciones_df = secciones_df[~secciones_df['nombre_seccion'].str.contains('opinión', case=False)]

Columnistas de opinion por dia

In [8]:
url_dia_columns=('https://www.eltiempo.com//colombia')
dia_columns=requests.get(url_dia_columns)
dia_columns.status_code

tcolombia_soup=BeautifulSoup(dia_columns.text,'lxml')

In [9]:
def obtener_rutas(s_nota):

    nombres = []
    enlaces = []

    # Encuentra todos los elementos 'a' con la clase 'page-link'
    noticias = s_nota.find_all('a', class_='boton page-link')

    # Itera a través de las noticias y extrae los nombres y enlaces
    for noticia in noticias:
        nombre = noticia.text.strip()  # Extrae el nombre de la noticia
        enlace = url+noticia['href']  # Obtiene el enlace de la noticia
            
        nombres.append(nombre)
        enlaces.append(enlace)

    ##.-.------
    # Encuentra todos los elementos 'a' con la clase 'page-link'
    primera = s_nota.find_all('article', class_='col2 notas principal main-module-bk play-inline contenido-exclusivo-icon')

    # Itera a través de las noticias y extrae los nombres y enlaces
    for noticia in primera:
        nombre = noticia.find('h3', class_='titulo').text.strip()
        enlace = url+noticia.find('a', class_='elecciones_titulo_uno')['href']
            
        nombres.append(nombre)
        enlaces.append(enlace)

    ##.-.------
    # Encuentra todos los elementos 'a' con la clase 'page-link'
    segunda = s_nota.find_all('article', class_='main')

    # Itera a través de las noticias y extrae los nombres y enlaces
    for noticia1 in segunda:
        nombre = noticia1.find('a', class_='title page-link').text.strip()
        enlace = url+noticia1.find('a', class_='title page-link')['href']
            
        nombres.append(nombre)
        enlaces.append(enlace)

    ##.-.------
    # Encuentra todos los elementos 'a' con la clase 'page-link'
    tercera = s_nota.find_all('article', class_='image-left')

    # Itera a través de las noticias y extrae los nombres y enlaces
    for noticia1 in tercera:
        nombre = noticia1.find('a', class_='title page-link').text.strip()
        enlace = url+noticia1.find('a', class_='title page-link')['href']
            
        nombres.append(nombre)
        enlaces.append(enlace)

    # Crear un DataFrame
    data = {'Nombre': nombres, 'Enlace': enlaces}
    noticias_df = pd.DataFrame(data)
    return noticias_df

In [10]:
df=obtener_rutas(tcolombia_soup)

------

Extraccion de informacion de noticias

In [11]:
url_nota=('https://www.eltiempo.com/colombia/otras-ciudades/temblor-en-colombia-balance-de-danos-en-el-calvario-meta-tras-sismo-797488')

In [22]:
def extaer_informacion(df):

    nota_nombre=[]
    nota_enlace=[]
    nota_encab=[]
    nota_fecha=[]
    nota_cuerpo=[]
    nota_imagen=[]

    for index in range(len(df)):
        url_nota=df.loc[index, 'Enlace']
        titular = df.loc[index, 'Nombre']
        nota_nombre.append(titular)

        try:
            
            nota=requests.get(url_nota)
            if nota.status_code==200:
                s_nota=BeautifulSoup(nota.text,'lxml')

                #Extraer el resumen
                encabezado=s_nota.find('div',class_='epigraph-container lead')
                nota_encab.append(encabezado.text)

                #Extraer fecha y hora de publicacion
                fecha=s_nota.find('div',class_='img_info h-seccion')
                fecha=fecha.find('span',class_='publishedAt').text
                nota_fecha.append(fecha)
                
                cuerpo1=""
                cuerpo=s_nota.find('div',class_='articulo-contenido')
                cuerpo=cuerpo.find_all('p',class_='contenido')
                cuerpo1=" ".join([p.get_text().strip() for p in cuerpo])
                nota_cuerpo.append(cuerpo1)
            
                try:
                    media =url+s_nota.find('div', class_='imagen-cuerpo-bk').img.get('src')
                    img_req = requests.get(media)
                    if img_req.status_code == 200:
                        nota_imagen.append(media)
                    else:
                        media =None
                        nota_imagen.append(media)
                except:
                    media =None
                    nota_imagen.append(media)
        except Exception as e:
            print('Error:')
            print(e)
            print('\n')
        # Crear un DataFrame
        data = {'titular':nota_nombre,'encabezado':nota_encab,'fecha':nota_fecha,'cuerpo':nota_cuerpo,'imagen':nota_imagen}
        noticias_df = pd.DataFrame(data)
    return noticias_df

In [24]:
data=extaer_informacion(df)

In [30]:
data.to_csv('pru.csv', sep='|', index=False)